In [2]:
# Try another way to model LDA: 
# http://chrisstrelioff.ws/sandbox/2014/11/13/getting_started_with_latent_dirichlet_allocation_in_python.html
# To action:
# try test what topic you get for the first row
# Understand PC1 and 2 & relevance matrix

In [3]:
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt

%matplotlib inline

import pandas as pd
from sklearn import datasets

/Users/jiaminlim93/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df= pd.read_csv('/Users/jiaminlim93/Desktop/GA Data Science Course/project/project/loan_use.csv',header=0)

In [5]:
# Do a small group to reduce computational time
sentences=df['loan_use'][0:1000].tolist()

In [6]:
len(sentences)

1000

## Cleaning data

### Removing non-letters

In [7]:
import re

### Tokenisation

In [8]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
raw = sentences[1].lower()
tokens = tokenizer.tokenize(raw)

In [9]:
tokens

['purchase', 'hygiene', 'products', 'and', 'round', 'out', 'his', 'inventory']

### Remove stop words

In [10]:
!pip install stop-words

In [11]:
from stop_words import get_stop_words

# create English stop words list
en_stop = get_stop_words('en')

In [12]:
# remove stop words from tokens
stopped_tokens = [i for i in tokens if not i in en_stop]

print(stopped_tokens)

['purchase', 'hygiene', 'products', 'round', 'inventory']


### Stemming

In [13]:
from nltk.stem.porter import PorterStemmer

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [14]:
# stem token
stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
print(stemmed_tokens)

[u'purchas', u'hygien', u'product', u'round', u'inventori']


### Looping through every element in the list

In [15]:
#https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words
def description_to_words( raw_description ):
    # Function to convert a raw description to a string of words
    # The input is a single string (a raw loan use description), and 
    # the output is a single string (a preprocessed loan use description)
    #
    # 1. Remove non-letters
    letters_only = re.sub("[^a-zA-Z]", " ", raw_description)
    # 2.Tokenisation
    lower_letters_only = letters_only.lower()
    tokens = tokenizer.tokenize(lower_letters_only)
    # 3. Remove stop words
    stopped_tokens = [i for i in tokens if not i in en_stop]
    # 4. Stemming
    meaningful_words = [p_stemmer.stem(i) for i in stopped_tokens]
    return meaningful_words
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    # return( " ".join( meaningful_words ))   

In [16]:
clean_description = description_to_words(sentences[0])
print sentences[0]
print clean_description

Compra de ropa Jeans, blusas, camisas, pantalones y variedad para ni̱o.
[u'compra', u'de', u'ropa', u'jean', u'blusa', u'camisa', u'pantalon', u'y', u'variedad', u'para', u'ni', u'o']


In [17]:
# Get the number of reviews based on the dataframe column size
num_descriptions = len(sentences)

# Initialize an empty list to hold the clean reviews
clean_descriptions = []

# Loop over each description; create an index i that goes from 0 to the length of the loan use description list 
# Inlcudes how long it takes to run
print "Cleaning and parsing the loan use data...\n"
clean_descriptions = []
for i in xrange( 0, num_descriptions ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Description %d of %d\n" % ( i+1, num_descriptions )                                                                    
    clean_descriptions.append( description_to_words( sentences[i] ) )

Cleaning and parsing the loan use data...

Description 1000 of 1000



In [18]:
clean_descriptions

[[u'compra',
  u'de',
  u'ropa',
  u'jean',
  u'blusa',
  u'camisa',
  u'pantalon',
  u'y',
  u'variedad',
  u'para',
  u'ni',
  u'o'],
 [u'purchas', u'hygien', u'product', u'round', u'inventori'],
 [u'rent',
  u'shop',
  u'purchas',
  u'raw',
  u'materi',
  u'bicycl',
  u'small',
  u'cart'],
 [u'go', u'rent', u'new', u'shop', u'enlarg', u'busi'],
 [u'purchas', u'new', u'equip'],
 [u'buy', u'paint', u'brush', u'materi', u'elei', u'wood', u'busi'],
 [u'buy',
  u'divers',
  u'spare',
  u'part',
  u'compra',
  u'de',
  u'repuesto',
  u'diverso'],
 [u'busi', u'expans'],
 [u'expand', u'poultri', u'busi'],
 [u'addit',
  u'work',
  u'capit',
  u'buy',
  u'good',
  u'improv',
  u'kiosk',
  u'condit'],
 [u'purchas', u'vari', u'merchandis'],
 [u'purchas', u'addit', u'stock', u'haircar', u'product', u'accessori'],
 [u'buy', u'bag', u'maiz', u'gari'],
 [u'stock', u'small', u'groceri', u'store'],
 [u'purchas', u'fresh', u'fish'],
 [u'buy', u'thread', u'shoe', u'decor'],
 [u'purchas', u'plot', u'lan

## Creating a document-term matrix

In [19]:
# https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
! pip install gensim
from gensim import corpora, models

/Users/jiaminlim93/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
/Users/jiaminlim93/anaconda/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [20]:
#type(clean_descriptions)
#isinstance(clean_descriptions[2], unicode)
#dic = corpora.Dictionary(clean_descriptions.split())
dictionary = corpora.Dictionary(clean_descriptions)

In [21]:
corpus = [dictionary.doc2bow(description) for description in clean_descriptions]

In [22]:
print sentences[20]
print clean_descriptions[20]
print(corpus[20])

Nimia would like to purchase cheese and the ingredients to prepare chicken relleno to sell.
[u'nimia', u'like', u'purchas', u'chees', u'ingredi', u'prepar', u'chicken', u'relleno', u'sell']
[(13, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1)]


In [23]:
len(dictionary)

975

## LDA model

### Fitting model

In [24]:
from gensim import corpora, models
ldamodel = models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=10)
print(ldamodel.print_topics(num_topics=5, num_words=10))

[(0, u'0.074*"buy" + 0.058*"purchas" + 0.036*"store" + 0.031*"fish" + 0.026*"stock" + 0.021*"product" + 0.017*"sell" + 0.014*"rice" + 0.013*"sari" + 0.010*"cloth"'), (1, u'0.045*"materi" + 0.044*"purchas" + 0.030*"de" + 0.024*"buy" + 0.024*"build" + 0.021*"hous" + 0.019*"construct" + 0.011*"room" + 0.010*"wood" + 0.010*"achat"'), (2, u'0.120*"purchas" + 0.051*"sell" + 0.037*"addit" + 0.033*"buy" + 0.030*"product" + 0.025*"capit" + 0.020*"increas" + 0.020*"cloth" + 0.019*"busi" + 0.019*"work"'), (3, u'0.080*"purchas" + 0.049*"busi" + 0.033*"expand" + 0.027*"rice" + 0.025*"pay" + 0.022*"suppli" + 0.017*"invest" + 0.017*"cow" + 0.015*"sugar" + 0.014*"oil"'), (4, u'0.183*"buy" + 0.017*"busi" + 0.016*"farm" + 0.015*"fertil" + 0.013*"materi" + 0.011*"resel" + 0.010*"merchandis" + 0.010*"seed" + 0.010*"cloth" + 0.010*"product"')]


### Visualisation

In [28]:
#h ttps://github.com/bmabey/pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      28.421725        1       1 -0.120118 -0.054625
4      23.924284        1       2  0.042229 -0.113116
3      18.852751        1       3 -0.050505  0.175788
0      16.384916        1       4 -0.066790 -0.033072
1      12.416324        1       5  0.195184  0.025025, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
890   Default  341.000000         buy  341.000000  30.0000  30.0000
671   Default  314.000000     purchas  314.000000  29.0000  29.0000
849   Default   45.000000      materi   45.000000  28.0000  28.0000
444   Default   94.000000        sell   94.000000  27.0000  27.0000
395   Default   30.000000       store   30.000000  26.0000  26.0000
642   Default   55.000000       addit   55.000000  25.0000  25.0000
721   Default   33.000000      expand   33.000000  24.0000  24.0000
306   Default   30.000000        fish   30.000000  23.0000  23.0000
501   Default   96.000000        busi   96.000000  22.0000  22.0000
141   Default   36.000000       capit   36.000000  21.0000  21.0000
510   Default   23.000000          de   23.000000  20.0000  20.0000
690   Default   24.000000         pay   24.000000  19.0000  19.0000
504   Default   42.000000        rice   42.000000  18.0000  18.0000
452   Default   17.000000       build   17.000000  17.0000  17.0000
166   Default   26.000000        work   26.000000  16.0000  16.0000
209   Default   46.000000       stock   46.000000  15.0000  15.0000
163   Default   14.000000        hous   14.000000  14.0000  14.0000
385   Default   13.000000   construct   13.000000  13.0000  13.0000
518   Default   40.000000     increas   40.000000  12.0000  12.0000
589   Default   22.000000         cow   22.000000  11.0000  11.0000
965   Default   21.000000      invest   21.000000  10.0000  10.0000
687   Default   19.000000   inventori   19.000000   9.0000   9.0000
616   Default   41.000000      suppli   41.000000   8.0000   8.0000
210   Default   78.000000     product   78.000000   7.0000   7.0000
624   Default   10.000000        sari   10.000000   6.0000   6.0000
917   Default   13.000000       resel   13.000000   5.0000   5.0000
45    Default   22.000000        item   22.000000   4.0000   4.0000
10    Default   34.000000  merchandis   34.000000   3.0000   3.0000
39    Default   11.000000      school   11.000000   2.0000   2.0000
162   Default    7.000000        room    7.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
738    Topic5    3.359691           y    4.212291   1.8600  -5.1953
317    Topic5    2.667796           p    3.370660   1.8523  -5.4259
920    Topic5    2.664031      enlarg    3.370469   1.8509  -5.4273
225    Topic5    2.657912       short    3.370094   1.8488  -5.4296
510    Topic5   18.458969          de   23.548998   1.8426  -3.4916
311    Topic5    3.731570          et    5.046694   1.7843  -5.0903
204    Topic5    1.859499        tile    2.526008   1.7798  -5.7868
82     Topic5    1.859479      surtir    2.526008   1.7798  -5.7869
933    Topic5    1.857935          az    2.525993   1.7790  -5.7877
260    Topic5    1.856859          la    2.525982   1.7784  -5.7883
967    Topic5    1.854916      cabbag    2.525951   1.7774  -5.7893
136    Topic5    1.844822        type    2.525961   1.7719  -5.7948
96     Topic5    1.829976    daughter    2.525926   1.7639  -5.8028
133    Topic5    5.982324        wood    8.425447   1.7437  -4.6183
685    Topic5    3.540052       decor    5.053090   1.7303  -5.1430
849    Topic5   27.451708      materi   45.163962   1.5883  -3.0947
815    Topic5    5.191903        home    8.409361   1.6039  -4.7601
400    Topic5    3.334309        para    5.043801   1.6723  -5.2029
339    Topic5    4.722978      compra    8.364888   1.5146  -4.8547
338    Topic5    2.699665      cement    4.

In [29]:
# Might be able to convert it to html
pyLDAvis.save_html(vis, description)

ValueError: fileobj should be a filename or a writable file